## Inference notebook for Sledge Hack hackathon
- **AUTHOR**: Nikhil Kotra
- **AV_username**: nikhil1e9

In this notebook I have breakdown my approach step by step to solve **The Sledge Hack hackathon** conducted by **Analytics Vidhya** from 29-09-2023 to 08-10-2023 with full explanation showing all inferences made from the data and how a certain assumption is made based on the analysis of data. This notebook breaks down the solution from my solution notebook which can be found at https://github.com/nikhil-1e9/AV-Hackathon/blob/main/AV_match_prediction.ipynb 

***This notebook doesn't contain all the models present in the original solution notebook and focuses on only the weighted average model as I selected that model for my final predictions in the original solution. The main focus of this notebook is to explain the data driven decisions which I made and implemented in the solution notebook but wasn't able to explain them there due to time constraints.***

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
# Load data into csv files
train = pd.read_csv("C:/Users/relat/Downloads/data_zpyYWs0.csv")
sub = pd.read_csv("C:/Users/relat/Downloads/sample_submission_Gk1Mne1.csv")

In [3]:
train

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Pat Cummins,DNB,3,28,0,0,19 Oct 2011,v South Africa Centurion,1
1,2,Steve Smith,DNB,-,-,0,0,19 Oct 2011,v South Africa Centurion,1
2,10,Mitch Marsh,8*,1,19,1,0,19 Oct 2011,v South Africa Centurion,1
3,13,David Warner,20,-,-,0,0,19 Oct 2011,v South Africa Centurion,1
4,1,Pat Cummins,11*,1,73,0,0,23 Oct 2011,v South Africa Gqeberha,2
...,...,...,...,...,...,...,...,...,...,...
2570,22,Ravindra Jadeja,DNB,3,35,0,0,28 Oct 2009,v Australia Nagpur,553
2571,22,Ravindra Jadeja,DNB,2,41,0,0,31 Oct 2009,v Australia Delhi,554
2572,22,Ravindra Jadeja,23,0,44,1,0,5 Nov 2009,v Australia Hyderabad,555
2573,22,Ravindra Jadeja,57,0,36,0,0,8 Nov 2009,v Australia Guwahati,556


***train and sub data is same as provided for the competition.***

In [4]:
# sub

In [5]:
# Check data statictics
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2575 entries, 0 to 2574
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2575 non-null   int64 
 1   player_name    2575 non-null   object
 2   runs_scored    2575 non-null   object
 3   wickets        2575 non-null   object
 4   runs_conceded  2575 non-null   object
 5   catches        2575 non-null   object
 6   stumpings      2575 non-null   object
 7   match_date     2575 non-null   object
 8   opposition     2575 non-null   object
 9   match_id       2575 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 201.3+ KB


In [6]:
# clean runs scored and convert to int
train['runs_scored'] = train['runs_scored'].str.replace('*', '', regex=False)
train['runs_scored'].replace({'DNB': '-1', 'TDNB': '-1'}, inplace=True)
train['runs_scored'] = train['runs_scored'].astype('int')

# clean wickets and convert to int
train['wickets'] = train['wickets'].str.replace('-', '0', regex=False).astype('int')

# convert date to datetime
train['match_date'] = pd.to_datetime(train['match_date'])

# extract different ground names
train['ground'] = train['opposition'].apply(lambda x: x.split()[-1])

# extract opponent team
train['opposition'] = train['opposition'].apply(lambda x: x.split()[1])
train['opposition'].replace({'South': 'South Africa',
                             'New': 'New Zealand',
                             'West': 'West Indies',
                             'Sri': 'Sri Lanka',
                             'Hong': 'Hong Kong'}, inplace=True)

In [7]:
id_name = train.groupby(['player_id']).player_name.first().reset_index()
id_dict = id_name.set_index('player_id').to_dict()['player_name']
# id_dict

## Making inferences from data
In this section I tried to predict the potential playing XI of both teams based on past and historical records for runs socred, wickets taken, stumpings, matches played etc.

### AUSTRALIA

#### Matches

In [8]:
# Total matches played by a player 
aus_tot_mat = train.groupby(['player_id', 'player_name']).player_id.count()[:15]
aus_tot_mat.sort_values(ascending=False)

player_id  player_name       
13         David Warner          150
2          Steve Smith           145
11         Glenn Maxwell         129
15         Mitchell Starc.       111
14         Adam Zampa             85
10         Mitch Marsh            79
1          Pat Cummins            77
8          Josh Hazlewood         74
3          Alex Carey             71
12         Marcus Stoinis         64
9          Travis Head            58
6          Marnus Labuschagne     38
7          Cameron Green          20
5          Sean Abbott            17
4          Josh Inglis             8
Name: player_id, dtype: int64

Warner, Smith, Maxwell, Starc, Zampa, Marsh, Cummins, Hazlewood are important and experienced players for AUS.

#### Runs

In [9]:
# Average runs scored by player in all matches
aus_avg_run = train.groupby(['player_id', 'player_name']).runs_scored.mean()[:15]
aus_avg_run.sort_values(ascending=False)

player_id  player_name       
13         David Warner          42.633333
9          Travis Head           35.534483
2          Steve Smith           34.744828
6          Marnus Labuschagne    33.315789
10         Mitch Marsh           28.189873
11         Glenn Maxwell         27.007752
3          Alex Carey            25.464789
12         Marcus Stoinis        21.781250
7          Cameron Green         18.800000
4          Josh Inglis           17.875000
5          Sean Abbott           13.235294
1          Pat Cummins            4.363636
15         Mitchell Starc.        3.927928
14         Adam Zampa             2.270588
8          Josh Hazlewood         0.932432
Name: runs_scored, dtype: float64

In [10]:
# Total runs scored by player in all matches
aus_run_sum = train.groupby(['player_id', 'player_name']).runs_scored.sum()[:15]
aus_run_sum.sort_values(ascending=False)

player_id  player_name       
13         David Warner          6395
2          Steve Smith           5038
11         Glenn Maxwell         3484
10         Mitch Marsh           2227
9          Travis Head           2061
3          Alex Carey            1808
12         Marcus Stoinis        1394
6          Marnus Labuschagne    1266
15         Mitchell Starc.        436
7          Cameron Green          376
1          Pat Cummins            336
5          Sean Abbott            225
14         Adam Zampa             193
4          Josh Inglis            143
8          Josh Hazlewood          69
Name: runs_scored, dtype: int32

David Warner, Steve Smith, Glenn Maxwell, Mitch Marsh, Marnus Labuschagne, Alex Carey, Travis Head are potential options for batting

#### Wickets

In [11]:
# Average wickets taken by player in all matches
aus_wic = train.groupby(['player_id', 'player_name']).wickets.mean()[:15]
aus_wic.sort_values(ascending=False)

player_id  player_name       
15         Mitchell Starc.       1.981982
14         Adam Zampa            1.670588
1          Pat Cummins           1.636364
8          Josh Hazlewood        1.567568
5          Sean Abbott           1.176471
7          Cameron Green         0.800000
12         Marcus Stoinis        0.687500
10         Mitch Marsh           0.683544
11         Glenn Maxwell         0.496124
9          Travis Head           0.275862
2          Steve Smith           0.193103
6          Marnus Labuschagne    0.052632
3          Alex Carey            0.000000
4          Josh Inglis           0.000000
13         David Warner          0.000000
Name: wickets, dtype: float64

In [12]:
# Total wickets taken by player in all matches
aus_wic_sum = train.groupby(['player_id', 'player_name']).wickets.sum()[:15]
aus_wic_sum.sort_values(ascending=False)

player_id  player_name       
15         Mitchell Starc.       220
14         Adam Zampa            142
1          Pat Cummins           126
8          Josh Hazlewood        116
11         Glenn Maxwell          64
10         Mitch Marsh            54
12         Marcus Stoinis         44
2          Steve Smith            28
5          Sean Abbott            20
7          Cameron Green          16
9          Travis Head            16
6          Marnus Labuschagne      2
3          Alex Carey              0
4          Josh Inglis             0
13         David Warner            0
Name: wickets, dtype: int32

There needs to be a balance between batters and bowlers. Every good team must have a balance between the number of batters and bowlers to win a match. For bowlers Mitchell Starc, Adam Zampa, Pat Cummins, Josh Hazlewood have the best records. Maxwell, Marsh and Stoinis also have decent wickets.

#### Stumpings

In [13]:
# Total stumpings done by player in all matches
train['stumpings'] = train['stumpings'].str.replace('-', '-1', regex=False).astype('int')
aus_st_sum = train.groupby(['player_id', 'player_name']).stumpings.sum()[:15]
aus_st_sum.sort_values(ascending=False)

player_id  player_name       
3          Alex Carey            8
1          Pat Cummins           0
2          Steve Smith           0
4          Josh Inglis           0
5          Sean Abbott           0
6          Marnus Labuschagne    0
7          Cameron Green         0
8          Josh Hazlewood        0
9          Travis Head           0
10         Mitch Marsh           0
11         Glenn Maxwell         0
12         Marcus Stoinis        0
13         David Warner          0
14         Adam Zampa            0
15         Mitchell Starc.       0
Name: stumpings, dtype: int32

Alex Carey is the only wicketkeeper option. So has to be in the team.

We can combine the players in three categories based on the above data - Batsmen, Bowlers and All-rounders.
- **Batsmen** - David Warner, Steve Smith, Marnus Labuschagne, Alex Carey(wk), Travis Head are potential batsmen
- **All rounders** - Glenn Maxwell, Mitch Marsh, Cameron Green, Marcus Stoinis fall in this category and are potential all rounders.
- **Bowlers** - Mitchell Starc, Adam Zampa, Josh hazlewood, Pat Cummins(c) are good bowling options.

Similar process can be done for India to predict the potential playing XI.

### INDIA

#### Matches

In [14]:
# Total matches played by a player 
ind_tot_mat = train.groupby(['player_id', 'player_name']).player_id.count()[15:]
ind_tot_mat.sort_values(ascending=False)

player_id  player_name        
19         Virat Kohli            281
16         Rohit Sharma           251
22         Ravindra Jadeja        186
28         Ravichandran Ashwin    115
27         Mohammed Shami          94
26         Kuldeep Yadav           90
17         Hardik Pandya           82
24         Jasprit Bumrah          78
21         KL Rahul                61
20         Shreyas Iyer            47
23         Shardul Thakur          44
18         Shubman Gill            35
25         Mohammed Siraj          30
30         Suryakumar Yadav        30
29         Ishan Kishan            25
Name: player_id, dtype: int64

Kohli, Rohit, Jadeja, Ashwin are senior players with more experience. Gill, Siraj, Surya and Ishan are newer ones.

#### Runs

In [15]:
# Average runs scored by player in all matches
ind_avg_run = train.groupby(['player_id', 'player_name']).runs_scored.mean()[15:]
ind_avg_run.sort_values(ascending=False)

player_id  player_name        
18         Shubman Gill           54.771429
19         Virat Kohli            46.516014
16         Rohit Sharma           40.254980
20         Shreyas Iyer           38.212766
21         KL Rahul               37.508197
29         Ishan Kishan           35.320000
30         Suryakumar Yadav       22.166667
17         Hardik Pandya          21.170732
22         Ravindra Jadeja        13.854839
23         Shardul Thakur          7.045455
28         Ravichandran Ashwin     5.695652
27         Mohammed Shami          1.702128
26         Kuldeep Yadav           1.266667
25         Mohammed Siraj          0.666667
24         Jasprit Bumrah          0.230769
Name: runs_scored, dtype: float64

In [16]:
# Total runs scored by player in all matches
ind_run_sum = train.groupby(['player_id', 'player_name']).runs_scored.sum()[15:]
ind_run_sum.sort_values(ascending=False)

player_id  player_name        
19         Virat Kohli            13071
16         Rohit Sharma           10104
22         Ravindra Jadeja         2577
21         KL Rahul                2288
18         Shubman Gill            1917
20         Shreyas Iyer            1796
17         Hardik Pandya           1736
29         Ishan Kishan             883
30         Suryakumar Yadav         665
28         Ravichandran Ashwin      655
23         Shardul Thakur           310
27         Mohammed Shami           160
26         Kuldeep Yadav            114
25         Mohammed Siraj            20
24         Jasprit Bumrah            18
Name: runs_scored, dtype: int32

Virat Kohli, Rohit Sharma, Ravindra Jadeja, KL Rahul, Shubman Gill, Shreyas Iyer, Hardik Pandya, Ishan Kishan are potential options for batting.

#### Wickets

In [17]:
# Average wickets taken by player in all matches
ind_wic = train.groupby(['player_id', 'player_name']).wickets.mean()[15:]
ind_wic.sort_values(ascending=False)

player_id  player_name        
27         Mohammed Shami         1.819149
25         Mohammed Siraj         1.800000
26         Kuldeep Yadav          1.688889
24         Jasprit Bumrah         1.653846
23         Shardul Thakur         1.431818
28         Ravichandran Ashwin    1.347826
22         Ravindra Jadeja        1.096774
17         Hardik Pandya          0.963415
16         Rohit Sharma           0.031873
19         Virat Kohli            0.014235
18         Shubman Gill           0.000000
20         Shreyas Iyer           0.000000
21         KL Rahul               0.000000
29         Ishan Kishan           0.000000
30         Suryakumar Yadav       0.000000
Name: wickets, dtype: float64

In [18]:
# Total wickets taken by player in all matches
ind_wic_sum = train.groupby(['player_id', 'player_name']).wickets.sum()[15:]
ind_wic_sum.sort_values(ascending=False)

player_id  player_name        
22         Ravindra Jadeja        204
27         Mohammed Shami         171
28         Ravichandran Ashwin    155
26         Kuldeep Yadav          152
24         Jasprit Bumrah         129
17         Hardik Pandya           79
23         Shardul Thakur          63
25         Mohammed Siraj          54
16         Rohit Sharma             8
19         Virat Kohli              4
18         Shubman Gill             0
20         Shreyas Iyer             0
21         KL Rahul                 0
29         Ishan Kishan             0
30         Suryakumar Yadav         0
Name: wickets, dtype: int32

Ravindra Jadeja, Mohammed Shami, Ashwin, Kuldeep Yadav, Bumrah and Pandya have decent amount of wickets. Mohammed Siraj has a also a very good average so he can also be included in the team.

#### Stumpings

In [19]:
# Total stumpings done by player in all matches
# train['stumpings'] = train['stumpings'].str.replace('-', '-1', regex=False).astype('int')
ind_st_sum = train.groupby(['player_id', 'player_name']).stumpings.sum()[15:]
ind_st_sum.sort_values(ascending=False)

player_id  player_name        
21         KL Rahul               4
29         Ishan Kishan           1
23         Shardul Thakur        -1
24         Jasprit Bumrah        -1
25         Mohammed Siraj        -1
27         Mohammed Shami        -1
28         Ravichandran Ashwin   -1
30         Suryakumar Yadav      -1
16         Rohit Sharma          -2
17         Hardik Pandya         -2
18         Shubman Gill          -2
20         Shreyas Iyer          -2
22         Ravindra Jadeja       -2
26         Kuldeep Yadav         -2
19         Virat Kohli           -3
Name: stumpings, dtype: int32

KL Rahul and Ishan Kishan are potential wicketkeeper options

From above inforamtion we can now segregate the players and make a potential team.
- **Batsmen** - Virat Kohli, Rohit Sharma(c), KL Rahul(wk), Ishan Kishan(wk), Shubman Gill, Shreyas Iyer are potential batsmen
- **All rounders** - Hardik Pandya, Ravindra Jadeja, Ravi Ashwin fall in this category and are potential all rounders.
- **Bowlers** - Kuldeep Yadav, Jasprit Bumrah, Mohammed Shami, Mohammed Siraj are good bowling options.

## Ground analysis

In [20]:
train.ground.unique()

array(['Centurion', 'Gqeberha', 'Durban', 'Belfast', "Lord's",
       'Melbourne', 'Sydney', 'Hobart', 'Auckland', 'Southampton',
       'Manchester', 'Leeds', 'Canberra', 'Brisbane', 'Perth', 'Adelaide',
       'Hamilton', 'Birmingham', 'Oval', 'Chennai', 'Gardens', 'Indore',
       'Bengaluru', 'Nagpur', 'Hyderabad', 'Ranchi', 'Mohali', 'Delhi',
       'Dhabi', 'Bristol', 'Nottingham', 'Taunton', 'Wankhede', 'Rajkot',
       'Paarl', 'Bloemfontein', 'Pallekele', '(RPS)', 'Cardiff',
       'Visakhapatnam', 'Ahmedabad', 'Mirpur', 'Hambantota', 'Harare',
       'Sharjah', '(DSC)', 'Wellington', 'Providence', 'Basseterre',
       'Bridgetown', 'Benoni', 'Johannesburg', 'Town', 'Potchefstroom',
       'Townsville', 'Cairns', 'Chester-le-Street', 'Lahore', 'Dambulla',
       'Pune', 'Jaipur', 'Edinburgh', 'Kingstown', 'Islet', 'Karachi',
       'Kanpur', 'Cuttack', 'Kingston', 'Bulawayo', 'Spain', 'Sound',
       'Dharamsala', 'Kochi', 'Napier', 'Fatullah', 'Guwahati',
       'Brabourne', 

In [21]:
# Average runs scored by a player on Chennai pitch
train[train.ground=='Chennai'].groupby(['player_id', 'player_name']).runs_scored.mean().sort_values(ascending=False)

player_id  player_name        
20         Shreyas Iyer           70.000000
17         Hardik Pandya          61.500000
10         Mitch Marsh            47.000000
19         Virat Kohli            42.125000
11         Glenn Maxwell          39.000000
3          Alex Carey             38.000000
18         Shubman Gill           37.000000
6          Marnus Labuschagne     28.000000
5          Sean Abbott            26.000000
13         David Warner           24.000000
16         Rohit Sharma           19.857143
21         KL Rahul               19.000000
9          Travis Head            19.000000
12         Marcus Stoinis         14.000000
28         Ravichandran Ashwin    13.333333
22         Ravindra Jadeja        12.666667
15         Mitchell Starc.        10.000000
1          Pat Cummins             9.000000
14         Adam Zampa              7.500000
27         Mohammed Shami          7.000000
25         Mohammed Siraj          3.000000
26         Kuldeep Yadav           1.666667
2

Steve Smith and Suryakumar Yadav can be seen at the bottom. This can happen because of them not playing many matches on this ground. Apart from that Shreyas Iyer, Virat Kohli, Hardik Pandya and Mitch Marsh have performed better on this pitch according to historical data.

In [22]:
# Total wickets taken by a player on Chennai pitch
train[train.ground=='Chennai'].groupby(['player_id', 'player_name']).wickets.sum()

player_id  player_name        
1          Pat Cummins            0
2          Steve Smith            0
3          Alex Carey             0
5          Sean Abbott            1
6          Marnus Labuschagne     0
9          Travis Head            0
10         Mitch Marsh            0
11         Glenn Maxwell          0
12         Marcus Stoinis         3
13         David Warner           0
14         Adam Zampa             5
15         Mitchell Starc.        0
16         Rohit Sharma           0
17         Hardik Pandya          5
18         Shubman Gill           0
19         Virat Kohli            0
20         Shreyas Iyer           0
21         KL Rahul               0
22         Ravindra Jadeja        3
24         Jasprit Bumrah         1
25         Mohammed Siraj         2
26         Kuldeep Yadav          5
27         Mohammed Shami         1
28         Ravichandran Ashwin    5
30         Suryakumar Yadav       0
Name: wickets, dtype: int32

Based on the data we can compare between wickets taken by spinners and pacers. Pacers are **Pandya, Siraj ,Shami and Bumrah**. Spinners are **Ashwin, Jadeja, Zampa and Kuldeep**

- Wickets by pacers - 5+1+2+1=9
- Wickets by spinners - 5+3+5+5=18

**This means that spinners has edge over pacers on this pitch.** Let's take a look at the average too for more accurate numbers.

In [23]:
# Average wickets taken by a player on Chennai pitch
train[train.ground=='Chennai'].groupby(['player_id', 'player_name']).wickets.mean()

player_id  player_name        
1          Pat Cummins            0.000000
2          Steve Smith            0.000000
3          Alex Carey             0.000000
5          Sean Abbott            1.000000
6          Marnus Labuschagne     0.000000
9          Travis Head            0.000000
10         Mitch Marsh            0.000000
11         Glenn Maxwell          0.000000
12         Marcus Stoinis         1.500000
13         David Warner           0.000000
14         Adam Zampa             2.500000
15         Mitchell Starc.        0.000000
16         Rohit Sharma           0.000000
17         Hardik Pandya          2.500000
18         Shubman Gill           0.000000
19         Virat Kohli            0.000000
20         Shreyas Iyer           0.000000
21         KL Rahul               0.000000
22         Ravindra Jadeja        1.000000
24         Jasprit Bumrah         1.000000
25         Mohammed Siraj         2.000000
26         Kuldeep Yadav          1.666667
27         Mohammed Sha

- Average wickets by pacers - 2.5+1+2+0.5 = 6.00
- Average wickets by spinners - 2.5+1+1.66+1.66 = 6.83

This also lead to the same conclusion. So it can be safely said that this pitch will likely be better for spin bolwing.

### Additional ground data

I have also extracted ground historical data from [espncricinfo](https://www.espncricinfo.com/) site which is analyzed below.

In [24]:
ground_df = pd.read_excel("C:/Users/relat/Downloads/Ground_stats.xlsx")
ground_df

,Team,Score,Overs,Target,RPO,Inns,Result,Column1,Opposition,Start Date,2
0,Australia,270/6,50.0,NaN,5.40,1,won,NaN,v India,1987-10-09,ODI # 453
1,India,269,49.5,271.0,5.39,2,lost,NaN,v Australia,1987-10-09,ODI # 453
2,Australia,235/9,50.0,NaN,4.70,1,won,NaN,v Zimbabwe,1987-10-13,ODI # 456
3,Zimbabwe,139,42.4,236.0,3.25,2,lost,NaN,v Australia,1987-10-13,ODI # 456
4,Australia,241/6,50.0,NaN,4.82,1,won,NaN,v West Indies,1989-10-21,ODI # 578
5,West Indies,142,40.3,242.0,3.50,2,lost,NaN,v Australia,1989-10-21,ODI # 578
6,West Indies,221,49.2,NaN,4.47,1,lost,NaN,v India,1994-10-23,ODI # 936
7,India,225/6,48.2,222.0,4.65,2,won,NaN,v West Indies,1994-10-23,ODI # 936
8,New Zealand,286/9,50.0,NaN,5.72,1,lost,NaN,v Australia,1996-03-11,ODI # 1080
9,Australia,289/4,47.5,287.0,6.04,2,won,NaN,v New Zealand,1996-03-11,ODI # 1080


This contains all the matches ever played on the Chennai pitch. Original data source is - https://stats.espncricinfo.com/ci/engine/ground/58008.html?class=2;template=results;type=aggregate;view=innings . I extracted the data from the site using excel and converted to dataframe. ***For original solution I did the calculation from the website table itself and didn't extracted to excel or dataframe format.***

***This data is used to calculate the average scores for both teams to get an idea of the total runs they can score in the match. I will be taking only the first innings scores as these are more important because the team can score as many runs playing first but in 2nd innings the runs are constrained to only how much runs the opponent team made so will not give an accurate estimate. Also matches with no result are excluded from the count.***

In [25]:
# All 1st innings scores for India in Chennai excluding no result matches
ind_chn = ground_df[(ground_df['Inns']==1)&(ground_df['Team']=='India')&(ground_df['Result']!='n/r')]
ind_chn

,Team,Score,Overs,Target,RPO,Inns,Result,Column1,Opposition,Start Date,2
18,India,268,48.0,NaN,5.58,1,lost,NaN,v West Indies,2007-01-27,ODI # 2487
32,India,268,49.1,NaN,5.45,1,won,NaN,v West Indies,2011-03-20,ODI # 3141
34,India,267/6,50.0,NaN,5.34,1,won,NaN,v West Indies,2011-12-11,ODI # 3224
36,India,227/6,50.0,NaN,4.54,1,lost,NaN,v Pakistan,2012-12-30,ODI # 3314
38,India,299/8,50.0,NaN,5.98,1,won,NaN,v South Africa,2015-10-22,ODI # 3698
40,India,281/7,50.0,NaN,5.62,1,won,NaN,v Australia,2017-09-17,ODI # 3910
42,India,287/8,50.0,NaN,5.74,1,lost,NaN,v West Indies,2019-12-15,ODI # 4221


In [26]:
ind_chn.Score.apply(lambda x: x.split('/')[0]).astype('int').mean()

271.0

In [27]:
# All 1st innings scores for Australia in Chennai excluding no result matches
aus_chn = ground_df[(ground_df['Inns']==1)&(ground_df['Team']=='Australia')&(ground_df['Result']!='n/r')]
aus_chn

,Team,Score,Overs,Target,RPO,Inns,Result,Column1,Opposition,Start Date,2
0,Australia,270/6,50.0,NaN,5.40,1,won,NaN,v India,1987-10-09,ODI # 453
2,Australia,235/9,50.0,NaN,4.70,1,won,NaN,v Zimbabwe,1987-10-13,ODI # 456
4,Australia,241/6,50.0,NaN,4.82,1,won,NaN,v West Indies,1989-10-21,ODI # 578
44,Australia,269,49.0,NaN,5.48,1,won,NaN,v India,2023-03-22,ODI # 4545


In [28]:
aus_chn.Score.apply(lambda x: x.split('/')[0]).astype('int').mean()

253.75

## Model: Weighted average by date

This model assigns a weight to every individual match_date of each player, with higher weight given to recent matches and lower weight given to older matches. This model tries to capture the form of the player as the runs he scored 5 or 10 years ago don't matter much and the his recent performance depicts his form and he is more likely to perform likewise.

In [29]:
weights_df = train[train.player_id==1].sort_values('match_date')
weights_df['weight'] = [i/len(weights_df) for i in range(1,len(weights_df)+1)]
weights_df

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id,ground,weight
0,1,Pat Cummins,-1,3,28,0,0,2011-10-19,South Africa,1,Centurion,0.012987
4,1,Pat Cummins,11,1,73,0,0,2011-10-23,South Africa,2,Gqeberha,0.025974
7,1,Pat Cummins,6,1,49,0,0,2011-10-28,South Africa,3,Durban,0.038961
9,1,Pat Cummins,-1,1,11,0,0,2012-06-23,Ireland,4,Belfast,0.051948
12,1,Pat Cummins,4,1,53,0,0,2012-06-29,England,5,Lord's,0.064935
...,...,...,...,...,...,...,...,...,...,...,...,...
582,1,Pat Cummins,-1,2,22,1,0,2022-06-24,Sri Lanka,73,(RPS),0.948052
591,1,Pat Cummins,-1,3,62,0,0,2022-11-17,England,74,Adelaide,0.961039
601,1,Pat Cummins,-1,2,25,1,0,2022-11-22,England,75,Melbourne,0.974026
612,1,Pat Cummins,21,1,44,0,0,2023-09-22,India,76,Mohali,0.987013


As seen in above case the weights for the last match played by the player is 1 and keeps on decreasing for older matches. This is calculated as the inverse of the total matches played by the player multiplied by the match number for the specific row. In the above case total matches are 77(n) and each row number from the start (numbered from 1 to n) is multiplied by (1/77) to calclulate the weights for a particular match. After this the weighted average runs and wickets are calculated from the following formulae:
$$Wtd\;avg\;runs = \frac{\sum_{i=1}^n runs_i*wts_i}{\sum_{i=1}^n wts_i}$$ where $$ runs_i \neq -1$$
Similarly for wickets
$$Wtd\;avg\;wickets = \frac{\sum_{i=1}^n wickets_i*wts_i}{\sum_{i=1}^n wts_i}$$

In [30]:
# Functions to calculate weighted average runs and weighted average wickets
def weighted_mean_runs(df):
    return np.sum(df.runs_scored*df.weight)/np.sum(df.weight)
def weighted_mean_wickets(df):
    return np.sum(df.wickets*df.weight)/np.sum(df.weight)

In [31]:
# Making weighted predictions for all players and storing in dataframe
def make_wtd_predictions(df):
    runs = []
    wickets = []
    for i in range(1,31):
        weights_df = df[df.player_id==i].sort_values('match_date')
        weights_df['weight'] = [j/len(weights_df) for j in range(1,len(weights_df)+1)]
        
        # Excluding DNB runs scored which are given by -1
        weighted_runs = weighted_mean_runs(weights_df[(weights_df.player_id==i)&(weights_df.runs_scored!=-1)])
        runs.append(weighted_runs)

        weighted_wickets = weighted_mean_wickets(weights_df[weights_df.player_id==i])
        wickets.append(weighted_wickets)

    sub_df = sub.copy()
    sub_df['runs'] = runs
    sub_df['wickets'] = wickets
    sub_df['name'] = id_name['player_name']
    sub_df = sub_df[['player_id', 'name', 'runs', 'wickets']]
    return sub_df

In [32]:
# Make predictions
sub = make_wtd_predictions(train)
sub

,player_id,name,runs,wickets
0,1,Pat Cummins,8.047170,1.607060
1,2,Steve Smith,42.301579,0.051488
2,3,Alex Carey,27.164066,0.000000
3,4,Josh Inglis,20.000000,0.000000
4,5,Sean Abbott,17.656250,1.241830
5,6,Marnus Labuschagne,35.827004,0.045884
6,7,Cameron Green,20.910112,0.904762
7,8,Josh Hazlewood,5.553525,1.541982
8,9,Travis Head,40.410984,0.219755
9,10,Mitch Marsh,30.564077,0.579114


In [33]:
# This function calculates the total runs and wickets by top 11 batsman and bowlers for both teams
def sanity_check(df):
    # Making sure the sum of scores of best 11 batsmen for AUS is around 250-300 range and wickets should not exceed 10
    print("Total score for AUS by top 11 batsman:", df[:15].sort_values('runs', ascending=False)[:11].runs.sum())
    print("Total wickets taken by AUS top 11 bowlers:", df[:15].sort_values('wickets', ascending=False)[:11].wickets.sum())
    # Making sure the sum of scores of best 11 batsmen for IND is around 250-300 range and wickets should not exceed 10
    print("Total score for IND by top 11 batsman:", df[15:].sort_values('runs', ascending=False)[:11].runs.sum())
    print("Total wickets taken by IND top 11 bowlers:", df[15:].sort_values('wickets', ascending=False)[:11].wickets.sum())

The sanity_check function above gives an idea about the predicted runs and wickets to decide whether these need to be balanced out later on.

In [34]:
sanity_check(sub)

Total score for AUS by top 11 batsman: 330.1287559651316
Total wickets taken by AUS top 11 bowlers: 11.073605156231114
Total score for IND by top 11 batsman: 374.2490567534829
Total wickets taken by IND top 11 bowlers: 11.78374334781467


These scores seem to be far ahead than the normal range. The function considers the scores and wickets of the top 11 batsmen and top 11 bowlers. This would not be the case in real as not all batsman and all bowlers are playing(there would be a mix of both) therefore these would automatically drop down when we consider the selected 11 players from each team.

## Final teams
Based on the [inferences section](#Making-inferences-from-data) above I came to the conclusion about the potential players who can be playing in the match based on their past records. Australian team include 13 players but we have to drop two of them to make correct predictions. There is a choice between Cameron Green, Travis Head and Marcus Stoinis as other 10 players selected are either experienced or very good players according to historical data (rest of the players like Sean Abbott etc. don't have good historical records). Past records are more in favor of Travis Head as he has more runs and better average than both Cameron Green and Stoinis. But as I was following the news about the match I found out that Travis Head would likely not be playing because of his injury, so this leaves Stoinis and Green. As the model is predicting Green(20.91 runs, 0.90 wickets) to perform better in both batting as well as bowling than Stoinis(18.79 runs, 0.64 wickets), I will select Cameron Green.

For Indian team I shortlisted 13 of the 15 available players above who are likely to be playing based on the analysis above. The team contain 4 fast bowlers and 3 spinners. The pitch is more likely to favor spinners based on historical data therefore, of the 4 fast bowlers one can be dropped. Hardik Pandya is all rounder and can't be dropped, Jasprit Bumrah's stats are the best among all so he is the best option. The choice now boils down between Mohammed Shami or Mohammed Siraj. This is a tough choice. As the weighted average model, which give more weight to the recent matches, is predicting Siraj to perform better(Siraj-1.96 wickets, Shami-1.84 wickets) then he must be in better form than Shami. Therefore I will be picking Siraj over Shami. For the last choice Ishan Kishan can be dropped as KL Rahul fulfils his role as wicketkeeper batsman and has a much better track record. But there is news of Shubman Gill also not playing as he is sick so only one of Ishan Kishan and Gill needs to be dropped. As the odds are against Gill who would likely not be playing which leaves Surya KumarYadav as a choice to consider but as far as my knowledge about cricket goes he is not good option because of his batting order. India would likely be in need of top order batsman if Shubman Gill doesn't play and Ishan Kishan fulfils that role. My model also is in favor of Ishan Kishan(38.81 runs) as his predicted runs are greater than Surya's(22.21 runs). so I will go with Ishan Kishan over Surya.

This leads to the potential playing XI of both teams which is given below:

- **AUS potential playing XI** - 1:Pat Cummins, 2:Steve Smith, 3:Alex Carey, 6:Marnus Labuschagne, 7:Cameron Green, 8:Josh Hazlewood, 10:Mitch Marsh, 11:Glenn Maxwell, 13:David Warner, 14:Adam Zampa, 15:Mitchell Starc

- **IND potential playing XI** - 16:Rohit Sharma, 17:Hardik Pandya, 19:Virat Kohli, 20:Shreyas Iyer, 21:KL Rahul, 22:Ravindra Jadeja, 24:Jasprit Bumrah, 25:Mohammed Siraj, 26:Kuldeep Yadav, 28:Ravichandran Ashwin, 29:Ishan Kishan

For the excluded players we don't have to include the predictions and in the post processing step we can change these predictions to zeros. 

- **Excluded Players**: 4: Josh Inglis, 5: Sean Abbott, 9: Travis Head, 12: Marcus Stoinis, 18: Shubman Gill, 23: Shardul Thakur, 27: Mohammed Shami, 30: Suryakumar Yadav


## Post processing and Final predictions
In this section, I will process the predictions from the weighted average model, calculated above, one final time depending on the playing 11 of both teams. Therefore only 22 columns will contain non-zero values and the 8 columns for players not playing will be zero. Also multiplying final predictions with a final weight score to balance the total score. The weight score is calculated from the historical performance of India on the Chennai pitch as calculated in [ground analysis section](#Additional-ground-data) above. This is also explained below.

In [35]:
# ids of players likely to be excluded
zero_id = [4, 5, 9, 12, 18, 23, 27, 30]

In [36]:
# Make columns zero for players not playing
sub.drop('name', axis=1, inplace=True)
for i in range(1, 31):
    if i in zero_id:
        sub.iloc[i-1,1:] = [0, 0]
sub

,player_id,runs,wickets
0,1,8.047170,1.607060
1,2,42.301579,0.051488
2,3,27.164066,0.000000
3,4,0.000000,0.000000
4,5,0.000000,0.000000
5,6,35.827004,0.045884
6,7,20.910112,0.904762
7,8,5.553525,1.541982
8,9,0.000000,0.000000
9,10,30.564077,0.579114


In [37]:
sanity_check(sub)

Total score for AUS by top 11 batsman: 260.2427496120536
Total wickets taken by AUS top 11 bowlers: 9.014635271208288
Total score for IND by top 11 batsman: 294.1139203387357
Total wickets taken by IND top 11 bowlers: 8.495287682539297


***Australia's predicted total came out to be around 260 from the model. The average score for Australia as calculated from historical ground data was around 253.75. As these scores are close there is no need for processing.***

***India's predicted total score came out to be around 294 from the model. The average score for India as calculated from historical ground data was around 271. The predicted score looks a little off from the average so I introduced a weight factor which is (India's avg score in Chennai/Total predicted score from model) and multiplied it to the predicted runs of Indian team.
As I calculated average score in Chennai by hand for the original solution directly from the website table I was getting 270.75 which is shown below. This happened because I accidentally also included the 1st entry which was 2nd Innings score for India and average becomes (269+268+268+267+227+299+281+287)/8 = 270.75.***


In [38]:
ground_df[(ground_df['Inns']==1)&(ground_df['Team']=='India')&(ground_df['Result']!='n/r')|(ground_df.index==1)]

,Team,Score,Overs,Target,RPO,Inns,Result,Column1,Opposition,Start Date,2
1,India,269,49.5,271.0,5.39,2,lost,NaN,v Australia,1987-10-09,ODI # 453
18,India,268,48.0,NaN,5.58,1,lost,NaN,v West Indies,2007-01-27,ODI # 2487
32,India,268,49.1,NaN,5.45,1,won,NaN,v West Indies,2011-03-20,ODI # 3141
34,India,267/6,50.0,NaN,5.34,1,won,NaN,v West Indies,2011-12-11,ODI # 3224
36,India,227/6,50.0,NaN,4.54,1,lost,NaN,v Pakistan,2012-12-30,ODI # 3314
38,India,299/8,50.0,NaN,5.98,1,won,NaN,v South Africa,2015-10-22,ODI # 3698
40,India,281/7,50.0,NaN,5.62,1,won,NaN,v Australia,2017-09-17,ODI # 3910
42,India,287/8,50.0,NaN,5.74,1,lost,NaN,v West Indies,2019-12-15,ODI # 4221


***I took the integer values of 294.11 -> 294 and 270.75 -> 270. This is why I multiplied the predicted runs for India by 270/294 in the original solution to balance out the predictions which can also be seen below.***

In [39]:
# Balancing predictions by multiplying by weight factor
sub.runs[15:] = sub.runs[15:]*270/294

In [40]:
sub

,player_id,runs,wickets
0,1,8.047170,1.607060
1,2,42.301579,0.051488
2,3,27.164066,0.000000
3,4,0.000000,0.000000
4,5,0.000000,0.000000
5,6,35.827004,0.045884
6,7,20.910112,0.904762
7,8,5.553525,1.541982
8,9,0.000000,0.000000
9,10,30.564077,0.579114


## Final Submission

In [41]:
sub.to_csv("submission.csv", index=False)

These are the exact same predictions as generated by the original code file as well as that were submitted as part of the final solution to the competition. Steps taken for verification of the same:
- Went to "My Submissions" tab in the Sledge Hack Hackathon and downloaded the Final predictions solution file
- Saved the file as last_predictions.csv locally and compared that with this final submission as well as the final predictions generated from the [original](https://github.com/nikhil-1e9/AV-Hackathon/blob/main/AV_match_prediction.ipynb) code.

The comparison is shown below:

## Comparison of all 3 files

In [42]:
# av_sub = pd.read_csv("last_predictions.csv")
# origin_sub = pd.read_csv("match_preds_final.csv")
# sub = pd.read_csv("submission.csv")

**av_sub** is the file downloaded from My Submissions, **origin_sub** is the file generated from [original](https://github.com/nikhil-1e9/AV-Hackathon/blob/main/AV_match_prediction.ipynb) and **sub** is generated in this notebook itself.

In [43]:
# av_sub==origin_sub

,player_id,runs,wickets
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True
7,True,True,True
8,True,True,True
9,True,True,True


In [44]:
# (av_sub == origin_sub).sum()

player_id    30
runs         30
wickets      30
dtype: int64

In [45]:
# (av_sub == sub).sum()

player_id    30
runs         30
wickets      30
dtype: int64

In [46]:
# (sub==origin_sub).sum()

player_id    30
runs         30
wickets      30
dtype: int64